In [ ]:
using CUDA
using KernelAbstractions
using CUDAKernels
using Juliana
using JSON

# Config

In [ ]:
data_dir = "/data/user/bellotti_r/data"
patient_ID = "test_01";

In [ ]:
gantry_angle = [70f0]
couch_angle = [30f0];

# Load data

In [ ]:
ct_path, patient = Juliana.load_patient_data(data_dir, patient_ID);

In [ ]:
hu_to_sp_dict = nothing
open("/data/user/bellotti_r/semester_project_planning_metrics/src/pyftpp/bin/huToSp.json") do file
    global hu_to_sp_dict = JSON.parse(file)
end

In [ ]:
@assert hu_to_sp_dict["dz"] == 1
@assert hu_to_sp_dict["z0"] == -1000

function convert_to_sp(value)
    return hu_to_sp_dict["densities"][value + 1001]
end;

In [ ]:
densities = convert_to_sp.(patient.ct.data)
d_densities = cu(densities);

# Calculate WED

In [ ]:
d_direction = cu(Juliana.angles_to_direction(gantry_angle, couch_angle))

In [ ]:
target_mask = Juliana.calculate_whole_target_mask(
    patient.prescriptions,
    patient.structures,
)
target_points = Juliana.mask_to_points(
    patient.ct.grid,
    target_mask,
)
d_target_points = cu(target_points);

In [ ]:
tmp = zeros(patient.ct.grid.size...)
tmp[80:160, 80:160, 30:40] .= 1
# tmp[97:154, 51:103, 30:40] .= 1
tmp = (tmp .== 1) .&& (Juliana.build_checker_board_mask(patient.ct.grid) .== 1)

eval_points, indices = Juliana.mask_to_points_and_indices(
   patient.ct.grid,
   tmp,
)
d_eval_points = cu(eval_points)

In [ ]:
d_points = d_eval_points
N = size(d_points, 2)

d_grid = cu(patient.ct.grid)
d_wed = cu(zeros(Float32, N));

calculate_wed_simple = Juliana.wed_kernel(CUDADevice(), 32);
event = calculate_wed_simple(
    d_wed,
    d_densities,
    d_grid,
    d_points,
    d_direction,
    ndrange=(N, 1),
#     STEP_SIZE=0.005,
)
wait(event);

wed_flat = collect(d_wed);

# Calculate Fiona WED

In [ ]:
working_dir = "tmp_wed_simple"
mkpath(working_dir)

In [ ]:
fiona_standalone_bin_path = "/data/user/bellotti_r/semester_project_planning_metrics/src/pyftpp/bin"
fiona_jar_path = "$fiona_standalone_bin_path/ch.psi.ftpp.standalone.planner-1.0.7.jar";

In [ ]:
optim_grid = Juliana.get_optimisation_grid(
    eval_points',
    patient.ct.grid,
)

In [ ]:
Dij, optim_points = Juliana.FionaStandalone.calculate_Dij(
    working_dir,
    ct_path,
    1,
    patient.structures[Juliana.hottest_target(patient.prescriptions)[1]],
    fiona_standalone_bin_path,
    fiona_jar_path,
    optim_grid,
    gantry_angle,
    couch_angle,
    [15.0f0],
    log_wed=true,
    optimization_points=collect(d_points)',
);

In [ ]:
size(optim_points)

In [ ]:
using JSON

open("$(working_dir)/WED_0.json") do file
    global wed_fiona_flat = JSON.parse(file)
end

# Convert to 3D tensor

In [ ]:
wed_fiona = zeros(Float32, patient.ct.grid.size...)
indices = Juliana.point_to_grid_index(collect(d_points)', patient.ct.grid)
for (w, index) in zip(wed_fiona_flat, eachrow(indices))
    wed_fiona[index...] = w
end

wed = zeros(Float32, patient.ct.grid.size...)
for (w, index) in zip(wed_flat, eachrow(indices))
    wed[index...] = w
end

# Select only points within optimisation grid

This is needed because Fiona has memory issues if the optimisation grid is too big...

In [ ]:
optim_grid_mask = zeros(patient.ct.grid.size...)
start_index = convert.(Int64, floor.((optim_grid.origin .- patient.ct.grid.origin) ./ patient.ct.grid.spacing))
end_index = convert.(Int64, ceil.((optim_grid.origin .+ optim_grid.size .* optim_grid.spacing .- patient.ct.grid.origin) ./ patient.ct.grid.spacing))

optim_grid_mask[start_index[1]:end_index[1], start_index[2]:end_index[2], start_index[3]:end_index[3]] .= 1;

wed = wed .* optim_grid_mask
wed_fiona = wed_fiona .* optim_grid_mask;

# Plot WED

In [ ]:
maximum(wed .- wed_fiona)

In [ ]:
using PyPlot
fig, ax = PyPlot.subplots()
img = ax.imshow((wed .- wed_fiona)[60:180, 60:180, :30]', vmin=-3.3, vmax=3.3, cmap="coolwarm")
fig.colorbar(img)

In [ ]:
Juliana.plot_distributions(
    patient.ct,
    Dict("Simple (new)" => wed, "Fiona" => wed_fiona),
    [],
    60, 180,
    60, 180,
    30,
#     max(maximum(d_wed), maximum(wed_fiona)),
    25.2,
    Juliana.build_colorscheme(),
);

In [ ]:
argmax(wed)

# Export to DICOM

In [ ]:
patient_ID = "train_00"
new_patient_ID = "bellotti_r_dicom_test_00"
patient_name = "$(new_patient_ID)^$(new_patient_ID)"

study_instance_UID = Juliana.get_study_instance_uid(new_patient_ID)
frame_of_reference_UID = "$(study_instance_UID).0"
ct_series_instance_UID = "$(study_instance_UID).1"
structureset_series_instance_UID = "$(study_instance_UID).2"


output_dir = "dicom_test"
mkpath(output_dir)

In [ ]:
using DICOM

In [ ]:
ct_datasets = Juliana.ct_to_dicom(
    patient.ct,
    study_instance_UID,
    frame_of_reference_UID,
    ct_series_instance_UID,
    new_patient_ID,
    patient_name,
)
for ds in ct_datasets
    ds.SeriesDescription = "CT Images"
    dcm_write("$(output_dir)/CT.$(ds.InstanceNumber).dcm", ds)
end

In [ ]:
i = 0
series_instance_UID = "$(study_instance_UID).$(3+i)"

dicom_dose = Juliana.dose_to_dicom(
    Juliana.ScalarGrid(wed, patient.ct.grid),
    patient_name,
    new_patient_ID,
    study_instance_UID,
    series_instance_UID,
    frame_of_reference_UID,
)
DICOM.dcm_write("$(output_dir)/RT_dose_new.dcm", dicom_dose);

In [ ]:
i = 1
series_instance_UID = "$(study_instance_UID).$(3+i)"

dicom_dose = Juliana.dose_to_dicom(
    Juliana.ScalarGrid(wed_fiona, patient.ct.grid),
    patient_name,
    new_patient_ID,
    study_instance_UID,
    series_instance_UID,
    frame_of_reference_UID,
)
DICOM.dcm_write("$(output_dir)/RT_dose_fiona.dcm", dicom_dose);

In [ ]:
maximum(wed)

In [ ]:
maximum(wed_fiona)